# Building  Personalize ReRanking

## Imports 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like boto3(The AWS SDK) and Pandas/Numpy which are core data science tools

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
%store -r dataset_group_arn
%store -r useritems
%store -r bucket

## Create the Solution and Version

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.

### Select Recipe

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking" 
# aws-personalized-ranking selected for this solution. 

### Create and Wait for Solution

First you will create the solution with the API, then you will create a version. It will take several minutes to train the model and thus create your version of a solution. Once it gets started and you are seeing the in progress notifications it is a good time to take a break, grab a coffee, etc.

#### Create Solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-personal-ranking",   # Please change the solution name if you are changing the recipe
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

prr_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Create Solution Version

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = prr_solution_arn
)

prr_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### Wait for Solution Version to Have ACTIVE Status

This will take at least 20 minutes.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = prr_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more compelx datasets you should see improvements.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = prr_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is simply a hosted copy of your model. Again there will be a short wait so after executing you can take a quick break while the infrastructure is being provisioned.

#### Create Campaign

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-ranking-campaign",
    solutionVersionArn = prr_solution_version_arn,
    minProvisionedTPS = 1
)

prr_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Wait for Campaign to Have ACTIVE Status

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = prr_campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Get Sample ReRanking



In [ ]:
# Getting random itemlist
datasample = useritems.sample(n=20)
sampleitemlist = []
for index, row in datasample.iterrows():
    sampleitemlist.append(row[1])
    
    
#Getting a random user
user_id, item_id, _ = useritems.sample().values[0]

# print("USER: {}".format(item_id))
# print("ItemList:")
# print(sampleitemlist)

In [ ]:
# First load items into memory
allitemuri = f's3://{bucket}/allitem_w_meta.csv'
items = pd.read_csv(allitemuri, sep=',', usecols=[0,1], names=['asin', 'title'], index_col='asin',header=0)

# print(items)

def get_allstore_products(asin):
    """
    Takes in an ID, returns a title
    """
    asin = str(asin)
    return items.loc[asin]['title']


#### Call GetRecommendations

Using the user that you obtained above, the lines below will get recommendations for you and return the list of movies that are recommended.


In [ ]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn=prr_campaign_arn,
    inputList=sampleitemlist,
    userId = str(user_id)
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['personalizedRanking']


recommendation_list = []

for item in item_list:
    title = get_allstore_products(item['itemId'])
    recommendation_list.append(title)  
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Item Description'])
recommendations_df

## Review

Using the codes above you have successfully trained a deep learning model to generate Allstore Merchandise recommendations based on prior user behavior. Think about other types of problems where this data is available and what it might look like to build a system like this to offer those recommendations.

Now you are ready to move onto the next notebook `3.Building_Campaign_P-rank.ipynb`

